<a href="https://colab.research.google.com/github/mdhornstein/fossil-embeddings/blob/main/Fossil_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from IPython.display import display, Image, clear_output

In [ ]:
# Install wget and unzip if they aren't already available
!apt-get install wget unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# Download the reduced-FID dataset (approx. 1.8 GB)
!wget https://zenodo.org/records/6333970/files/reduced-FID.zip

--2025-08-23 19:26:53--  https://zenodo.org/records/6333970/files/reduced-FID.zip
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819654497 (1.7G) [application/octet-stream]
Saving to: ‘reduced-FID.zip’

reduced-FID.zip     100%[===================>]   1.69G  22.7MB/s    in 64s     

2025-08-23 19:27:58 (27.1 MB/s) - ‘reduced-FID.zip’ saved [1819654497/1819654497]



In [ ]:
!unzip -q reduced-FID.zip

In [ ]:
!ls

reduced-FID  reduced-FID.zip  sample_data


In [ ]:
# Verify the folder structure. You should see a folder named 'reduced-FID'.
# The images are organized into subfolders by clade (e.g., reduced-FID/Agnatha, reduced-FID/Amphibia)
!ls -l reduced-FID

total 4020
drwxr-xr-x 2 root root  94208 Jan  4  2022  agnatha
drwxr-xr-x 2 root root  81920 Jan  4  2022  ammonoid
drwxr-xr-x 2 root root  81920 Jan  4  2022  amphibian
drwxr-xr-x 2 root root  69632 Jan  4  2022  angiosperm
drwxr-xr-x 2 root root  81920 Jan  4  2022  avialae
drwxr-xr-x 2 root root  94208 Jan  4  2022  belemnite
drwxr-xr-x 2 root root  69632 Jan  4  2022  bivalve
drwxr-xr-x 2 root root  86016 Jan  4  2022  blastoid
drwxr-xr-x 2 root root  69632 Jan  4  2022  bone_fragment
drwxr-xr-x 2 root root  69632 Jan  4  2022  brachiopod
drwxr-xr-x 2 root root  73728 Jan  4  2022  bryozoan
drwxr-xr-x 2 root root  77824 Jan  4  2022  chelicerate
drwxr-xr-x 2 root root  73728 Jan  4  2022  chondrichthyes
drwxr-xr-x 2 root root  86016 Jan  4  2022  conodont
drwxr-xr-x 2 root root  69632 Jan  4  2022  coral
drwxr-xr-x 2 root root  81920 Jan  4  2022  crinoid
drwxr-xr-x 2 root root  86016 Jan  4  2022  crocodylomorph
drwxr-xr-x 2 root root  73728 Jan  4  2022  crustacean
drwxr-xr-x 2 r

In [ ]:
# --- Configuration ---
# Assuming 'reduced-FID' is directly in the Colab environment's root
DATASET_ROOT = './reduced-FID' # Changed to local path

## Ammonoid

In [ ]:
display(Image(filename=os.path.join(DATASET_ROOT, 'ammonoid', '001_1.jpg'), width=400))

## Theropod

In [ ]:
display(Image(filename=os.path.join(DATASET_ROOT, 'theropod', '001ec949fb1b0f3b1c0808.jpg'), width=400))

## Trilobite

In [ ]:
display(Image(filename=os.path.join(DATASET_ROOT, 'trilobite', '000061.jpg'), width=400))